# Sctructured Output for stream mode

In [1]:
from openai import OpenAI

In [2]:
openai_client = OpenAI()

In [ ]:
messages = [
    {"role": "user", "content": "Tell me a bas time story about a unicorn."},
]

stream = openai_client.responses.create(
    model = 'gpt-4o-mini',
    input = messages,
    stream = True
)

response = None

for event in stream:
    if hasattr(event, 'delta'):
        print(event.delta, end='')
    if hasattr(event, 'response'):
        response = event.response


**The Rainbow's End**

Once in the shimmering Valley of Lumina, where the skies were always painted in vibrant hues, lived a gentle unicorn named Liora. With a mane that glistened like stardust, Liora was beloved by all the animals in the valley. She had a special gift: wherever she trotted, flowers bloomed, and rainbows arched in the sky.

Despite her joyful spirit, Liora had one worry: she had never seen the end of a rainbow. Legends spoke of treasures and wonders found there, and Liora longed to experience this magic.

One day, after a rainstorm, a radiant rainbow appeared. With determination in her heart, Liora set off to find its end. She trotted through fields of daisies, over sparkling streams, and beneath towering trees, guided by the vibrant colors above her.

Along her journey, Liora met a wise old owl named Oliver. “Where are you off to, dear Liora?” he asked. She told him about her quest to find the end of the rainbow.

“Ah, the end of a rainbow is not a place, but a feelin

In [8]:
from pydantic import BaseModel, Field

In [9]:
class ArticleSection(BaseModel):
    header: str
    text: str = Field(description="text of the section in markdown")

class ArticleResponse(BaseModel):
    title: str
    subtitle: str
    sections: list[ArticleSection]


In [11]:
messages = [
    {"role": "user", "content": "Tell me a bas time story about a unicorn."},
]

with openai_client.responses.stream(
    model = 'gpt-4o-mini',
    input = messages,
    text_format= ArticleResponse
) as stream:
    response = None
    for event in stream:
        if hasattr(event, 'delta'):
            print(event.delta, end='')
        if hasattr(event, 'response'):
            response = event.response

{"title":"The Last Unicorn","subtitle":"A Magical Tale of Hope","sections":[{"header":"The Enchanted Forest","text":"Once upon a time, in a hidden part of the world, there was an enchanted forest where animals talked, trees danced, and magic flowed like a gentle river. In this mystical land lived the last unicorn named Lira. Lira was a magnificent creature with a shimmering coat as white as snow and a spiraled horn that sparkled in the sunlight."},{"header":"The Curse of Darkness","text":"However, not all was well in the enchanted forest. A dark sorcerer, envious of Lira’s beauty and magic, cast a curse that began to sap the forest of its colors. Flowers wilted, and the laughter of animals faded into silence. The once-vibrant forest was now shrouded in shadows, and Lira felt a deep sadness in her heart."},{"header":"Lira's Journey","text":"Determined to save her home, Lira embarked on a perilous journey to find the mythical Crystal of Light. It was said that the crystal could break any

In [13]:
story = response.output_parsed

In [17]:
print('# ' + story.title)
print(story.subtitle)
print()
for section in story.sections:
    print('## ' + section.header)
    print(section.text)
    print()
    print()

# The Last Unicorn
A Magical Tale of Hope

## The Enchanted Forest
Once upon a time, in a hidden part of the world, there was an enchanted forest where animals talked, trees danced, and magic flowed like a gentle river. In this mystical land lived the last unicorn named Lira. Lira was a magnificent creature with a shimmering coat as white as snow and a spiraled horn that sparkled in the sunlight.


## The Curse of Darkness
However, not all was well in the enchanted forest. A dark sorcerer, envious of Lira’s beauty and magic, cast a curse that began to sap the forest of its colors. Flowers wilted, and the laughter of animals faded into silence. The once-vibrant forest was now shrouded in shadows, and Lira felt a deep sadness in her heart.


## Lira's Journey
Determined to save her home, Lira embarked on a perilous journey to find the mythical Crystal of Light. It was said that the crystal could break any curse and restore life to the forest. Along the way, she faced many challenges—a ri

# use jaxn
install -> uv add jaxn

In [18]:
from jaxn import JSONParserHandler, StreamingJSONParser

In [26]:
from typing import Any, Dict

class ArticleResponseHandler(JSONParserHandler):

    def on_field_end(self, path: str, field_name: str, value: str, parsed_value: Any = None) -> None:
        if path == '':
            if field_name == 'title':
                print(f'# {value}')
            if field_name == 'subtitle':
                print(value)
        if path == '/sections' and field_name == 'header':
            print(f'\n\n## {value}\n')

    def on_value_chunk(self, path: str, field_name: str, chunk: str) -> None:
        if path == '/sections' and field_name == 'text':
            print(chunk, end='', flush=True)

In [27]:
handler = ArticleResponseHandler()
parser = StreamingJSONParser(handler = handler)

In [28]:
raw_json = """
{"title":"The Last Unicorn","subtitle":"A Magical Tale of Hope","sections":[{"header":"The Enchanted Forest","text":"Once upon a time, in a hidden part of the world, there was an enchanted forest where animals talked, trees danced, and magic flowed like a gentle river. In this mystical land lived the last unicorn named Lira. Lira was a magnificent creature with a shimmering coat as white as snow and a spiraled horn that sparkled in the sunlight."},{"header":"The Curse of Darkness","text":"However, not all was well in the enchanted forest. A dark sorcerer, envious of Lira’s beauty and magic, cast a curse that began to sap the forest of its colors. Flowers wilted, and the laughter of animals faded into silence. The once-vibrant forest was now shrouded in shadows, and Lira felt a deep sadness in her heart."},{"header":"Lira's Journey","text":"Determined to save her home, Lira embarked on a perilous journey to find the mythical Crystal of Light. It was said that the crystal could break any curse and restore life to the forest. Along the way, she faced many challenges—a river of fire, a mountain of ice, and a dark cave filled with echoes of despair. But Lira was brave and never gave up."},{"header":"Friendship and Hope","text":"During her quest, Lira met a clever fox named Finn and a wise old owl named Sage. They became her companions, supporting her with their wisdom and humor. Together, they learned that true magic lay not just in spells but in friendship, courage, and hope."},{"header":"The Final Confrontation","text":"Finally, Lira and her friends reached the sorcerer's lair, an eerie castle hidden beneath the roots of an ancient tree. Confronting the sorcerer, Lira used her unwavering spirit and the power of friendship to counter his dark magic. With a burst of light from her horn, she illuminated the castle, revealing the sorcerer’s true self—a lonely soul craving connection."},{"header":"The Transformation","text":"Touched by Lira's kindness, the sorcerer realized the emptiness of his envy. He lifted the curse, and color flooded back into the forest. Flowers bloomed, the sun shone brighter, and laughter echoed once more. The sorcerer, now humbled, chose to become the forest’s protector."},{"header":"A New Beginning","text":"From that day forward, Lira, Finn, Sage, and the reformed sorcerer worked together to heal the land. The enchanted forest thrived again, and though Lira was the last unicorn, she became a symbol of hope and love, reminding everyone that courage, friendship, and kindness can light even the darkest paths."}]}
""".strip()

In [29]:
for c in raw_json:
    parser.parse_incremental(c)

# The Last Unicorn
A Magical Tale of Hope


## The Enchanted Forest

Once upon a time, in a hidden part of the world, there was an enchanted forest where animals talked, trees danced, and magic flowed like a gentle river. In this mystical land lived the last unicorn named Lira. Lira was a magnificent creature with a shimmering coat as white as snow and a spiraled horn that sparkled in the sunlight.

## The Curse of Darkness

However, not all was well in the enchanted forest. A dark sorcerer, envious of Lira’s beauty and magic, cast a curse that began to sap the forest of its colors. Flowers wilted, and the laughter of animals faded into silence. The once-vibrant forest was now shrouded in shadows, and Lira felt a deep sadness in her heart.

## Lira's Journey

Determined to save her home, Lira embarked on a perilous journey to find the mythical Crystal of Light. It was said that the crystal could break any curse and restore life to the forest. Along the way, she faced many challenges—a 

## Example with RAG

In [36]:
def llm_structured_stream(
        user_prompt,
        output_type,
        parser_handler=JSONParserHandler(),
        instructions = None,
        model = 'gpt-4o-mini'
):
    messages = []

    if instructions:
        messages.append({
            "role": "system",
            "content": instructions
        })

    messages.append({
        "role": "user",
        "content": user_prompt,
    })

    parser = StreamingJSONParser(handler = parser_handler)

    with openai_client.responses.stream(
        model = model,
        input = messages,
        text_format= output_type
    ) as stream:
        response = None
        for event in stream:
            if hasattr(event, 'delta'):
                parser.parse_incremental(event.delta)
            if hasattr(event, 'response'):
                response = event.response
    
    return response

In [37]:
instructions = "your task is to tell the user bad time stories"
user_prompt = "unicorn"

result = llm_structured_stream(
    instructions=instructions,
    user_prompt=user_prompt,
    output_type=ArticleResponse,
    parser_handler=ArticleResponseHandler(),
)

# The Cursed Unicorn
A Tale of Misfortune and Regret


## The Enchanted Forest

In a mystical land far away, there existed an enchanted forest where unicorns roamed free. Among them was a radiant unicorn named Celestia, known for her dazzling white coat and silver mane. She was adored by all who encountered her, bringing joy and hope to the hearts of the villagers.

## The Greedy Wish

One fateful day, a greedy sorcerer wandered into the forest, seeking to capture Celestia. He believed that if he could obtain her magical horn, he could grant himself eternal youth and immense power. Overwhelmed by his desire, he devised a sinister plan.

## The Dark Trap

The sorcerer cast a series of dark spells to trap Celestia within enchanted vines. As she struggled to break free, she unintentionally unleashed a curse upon the forest. The once vibrant flowers wilted, and shadows began to stretch longer, casting a gloom over her home.

## The Cost of Greed

Despite her desperate attempts to escape, C

In [38]:
from gitsource import GithubRepositoryDataReader, chunk_documents
from minsearch import Index

reader = GithubRepositoryDataReader(
    repo_owner="evidentlyai",
    repo_name="docs",
    allowed_extensions={"md", "mdx"},
)
files = reader.read()

parsed_docs = [doc.parse() for doc in files]
chunked_docs = chunk_documents(parsed_docs, size=3000, step=1500)

index = Index(
    text_fields=["title", "description", "content"],
    keyword_fields=["filename"]
)
index.fit(chunked_docs)

print(f"Indexed {len(chunked_docs)} chunks from {len(files)} documents")

Indexed 385 chunks from 95 documents


In [39]:
def search(query):
    results = index.search(
        query=query,
        num_results=5
    )
    return results

In [40]:
import json

instructions = """
You're a documentation assistant. Answer the QUESTION based on the CONTEXT from our documentation.

Use only facts from the CONTEXT when answering.
If the answer isn't in the CONTEXT, say so.
"""

prompt_template = """
<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(question, search_results):
    context = json.dumps(search_results, indent=2)
    return prompt_template.format(
        question=question,
        context=context
    )

In [41]:
from typing import Literal

class RagResponse(BaseModel):
    """
    This model provides a structured answer with metadata about the response,
    including confidence, categorization, and follow-up suggestions.
    """
    answer: str = Field(description="The main answer to the user's question in markdown")
    found_answer: bool = Field(description="True if relevant information was found in the documentation")
    confidence: float = Field(description="Confidence score from 0.0 to 1.0 indicating how certain the answer is")
    confidence_explanation: str = Field(description="A brief explanation of the confidence score, highlighting key factors that influenced it")
    answer_type: Literal["how-to","explanation", "troubleshoting","comparison","reference"] = Field(description="The category of the answer")
    followup_questions: list[str] = Field(description="Suggested follow-up questions the user might want to ask")

In [55]:
class RagResponseHandler(JSONParserHandler):
    def on_value_chunk(self, path: str, field_name: str, chunk: str) -> None:
        if path == '' and field_name == 'answer':
            print(chunk, end='', flush=True)

    def on_field_end(self, path: str, field_name: str, value: str, parsed_value: Any = None) -> None:
        if path == '' and field_name == 'answer_type':
            print('\nanswer type:', value)

    def on_array_item_end(self, path: str, field_name: str, item: Dict[str, Any] = None) -> None:
        if field_name == 'followup_questions':
            print('follow up question:', item)

In [56]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)

    return llm_structured_stream(instructions = instructions,
                           user_prompt= prompt, 
                           output_type = RagResponse,
                           parser_handler=RagResponseHandler())

In [57]:
response = rag('llm as a judge')

## Using LLM as a Judge

In the context of evaluating text based on custom criteria, LLM (Language Model) serves as a judge by assessing the quality of responses to queries. Here’s a breakdown of how to implement an LLM as a judge:

### 1. **Evaluation Approaches**  
You can utilize an LLM judge in two key ways:
- **Reference-based**: This approach compares new responses against an established reference or "ground truth" for regression testing, ensuring that the new outputs align with the expected correct answers.
- **Open-ended**: This evaluation method establishes custom criteria for assessing responses, which is particularly useful when no predefined reference is available.

### 2. **Creating an Evaluation Dataset**  
A toy Q&A dataset can be created, which should include:
- Questions: Inputs to the LLM application  
- Target Responses: Approved correct responses  
- New Responses: Actual outputs from the system  
- Manual Labels: Indications of whether the responses are correct or 